In [ ]:
!pip install -U pyarrow

Requirement already up-to-date: pyarrow in /usr/local/lib/python3.6/dist-packages (2.0.0)


In [ ]:
!pip3 uninstall -y lightgbm

Uninstalling lightgbm-3.1.1:
  Successfully uninstalled lightgbm-3.1.1


In [ ]:
!apt-get install -y -qq libboost-all-dev

In [ ]:
!pip3 install lightgbm --install-option=--gpu --install-option="--opencl-include-dir=/usr/local/cuda/include/" --install-option="--opencl-library=/usr/local/cuda/lib64/libOpenCL.so" 

/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
  Using cached https://files.pythonhosted.org/packages/cf/65/368931f1f234149e14fc6075dbeb1ad8a8aebb105aa11fe8631c72c1bdcf/lightgbm-3.1.1.tar.gz
Skipping wheel build for lightgbm, due to binaries being disabled for it.
    Running setup.py install for lightgbm ... done


In [ ]:
import os
import sys
import time
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import gc
import random
from pickle import dump, load

pd.options.mode.chained_assignment = None

In [ ]:
import lightgbm as lgb

In [ ]:
def read_csv(file_name = "train.csv", dtype = None, skiprows = None, nrows = None, usecols = None):
    data = pd.read_csv(file_name, dtype=dtype, skiprows = skiprows, nrows = nrows, low_memory = True, header = 0, usecols = usecols)
    return data

def read_feather(file_name = "train.feather"):
    data = pd.read_feather(file_name)
    return data

In [ ]:
#tdf = read_feather("/content/drive/MyDrive/data/new_mu_sigma_tdf_with_mean_trueskill_hung.feather")
tdf = read_feather("/content/drive/MyDrive/data/new_mu_sigma_vdf_with_mean_trueskill_hung.feather")
vdf = tdf[(tdf.index < 3000000) | (tdf.index>=20000000)]
tdf = tdf[(tdf.index >= 3000000)&(tdf.index<20000000)]
gc.collect()
print(tdf.shape)
print(vdf.shape)

(17000000, 34)
(7271300, 34)


In [ ]:
tdf

,ts_delta,row_id,user_id,content_id,task_container_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,part,t1,num_tag,u_attempt_c,bundle_id,total_explained,10_recent_correctness,10_recent_mean_gap,mean_elapsed,mean_gap,this_bundle_elapsed_time_mean,u_chance,u_attempts,c_chance,c_attempts,u_part_chance,u_part_attempts,u_skill_chance,u_skill_attempts,t_chance,t_attempts,prev_cor,prev_t1,trueskill_probability,mu,sigma
3000000,77,12576352,272470622,5703,77,1,28,1,5,154,1,1,4272,60,0.799805,20028.900391,27,470305,32,0.577778,90,0.371493,50289,0.681818,66,0.635135,74,0.657172,10834,1,154,0.134007,0.427686,0.028305
3000001,37,85769960,1825993815,4219,4806,0,4,1,5,67,1,0,2788,6388,0.700195,25.400000,26,1002,26,0.585370,6425,0.520963,1288,0.576214,2657,0.595238,3864,0.609569,41572,1,62,0.411785,0.418031,0.020096
3000002,46,95687040,2029505536,824,151,1,14,1,2,130,4,1,825,132,0.600098,39.799999,20,4070,18,0.641975,162,0.581569,54463,0.750000,44,0.677966,59,0.676754,29151,1,153,0.792095,0.567458,0.023789
3000003,475,96633561,2050316518,3928,464,0,25,1,5,60,1,0,2497,695,0.500000,859.900024,47,401,20,0.819602,704,0.664650,25615,0.673203,153,0.743590,273,0.632002,57877,1,54,0.875335,0.542888,0.022235
3000004,44,72182079,1536722837,3946,6016,1,42,1,5,67,1,0,2515,8482,0.500000,63.099998,38,1739,23,0.693991,9369,0.694634,1025,0.611832,3161,0.643505,6174,0.609568,41573,1,71,0.789838,0.444840,0.020943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19999995,25,89471105,1901056520,1027,2760,1,18,1,2,143,4,0,1028,3335,0.899902,31.100000,35,8579,17,0.645573,3366,0.809622,6319,0.728653,527,0.720728,1264,0.693247,15279,1,134,0.825527,0.357594,0.020232
19999996,33,78838795,1677927147,8789,517,0,9,1,5,31,1,0,6266,532,0.399902,48.000000,32,54295,35,0.528440,545,0.337379,6509,0.522177,496,0.523364,535,0.574895,33511,0,62,0.080339,0.409130,0.020655
19999997,69,94213763,1999891715,1315,381,0,15,1,2,153,5,0,1316,369,0.600098,88.599998,26,56638,19,0.744361,399,0.363448,19805,0.899083,109,0.858268,127,0.720505,29370,1,35,0.332007,0.559108,0.020900
19999998,649383,35628062,760932330,2533,121,1,38,1,3,157,3,0,1779,249,0.799805,578431.812500,35,172126,20,0.727778,360,0.906717,3677,0.583333,24,0.699454,183,0.687078,17085,0,102,0.977524,0.452832,0.021403


In [ ]:
# vdf = read_feather("/content/drive/MyDrive/data/new_mu_sigma_vdf_with_mean_trueskill_hung.feather")

In [ ]:
TARGET = 'answered_correctly'
drop_cols = ['row_id', 'user_id', 'content_id', 'bundle_id', 'answered_correctly']
cat_cols = ['prior_question_had_explanation', 'part', 't1', 'num_tag', 'prev_cor', 'prev_t1']

In [ ]:
y_train = tdf[TARGET]
y_val = vdf[TARGET]
tdf.drop(drop_cols, axis=1, inplace=True)
vdf.drop(drop_cols, axis=1, inplace=True)
gc.collect()

0

In [ ]:
tdf.fillna(-1, inplace = True)
vdf.fillna(-1, inplace = True)

In [ ]:
feat_cols = list(tdf.columns)

In [ ]:
tdf

,ts_delta,task_container_id,prior_question_elapsed_time,prior_question_had_explanation,part,t1,num_tag,u_attempt_c,total_explained,10_recent_correctness,10_recent_mean_gap,mean_elapsed,mean_gap,this_bundle_elapsed_time_mean,u_chance,u_attempts,c_chance,c_attempts,u_part_chance,u_part_attempts,u_skill_chance,u_skill_attempts,t_chance,t_attempts,prev_cor,prev_t1,trueskill_probability,mu,sigma
3000000,77,77,28,1,5,154,1,1,60,0.799805,20028.900391,27,470305,32,0.577778,90,0.371493,50289,0.681818,66,0.635135,74,0.657172,10834,1,154,0.134007,0.427686,0.028305
3000001,37,4806,4,1,5,67,1,0,6388,0.700195,25.400000,26,1002,26,0.585370,6425,0.520963,1288,0.576214,2657,0.595238,3864,0.609569,41572,1,62,0.411785,0.418031,0.020096
3000002,46,151,14,1,2,130,4,1,132,0.600098,39.799999,20,4070,18,0.641975,162,0.581569,54463,0.750000,44,0.677966,59,0.676754,29151,1,153,0.792095,0.567458,0.023789
3000003,475,464,25,1,5,60,1,0,695,0.500000,859.900024,47,401,20,0.819602,704,0.664650,25615,0.673203,153,0.743590,273,0.632002,57877,1,54,0.875335,0.542888,0.022235
3000004,44,6016,42,1,5,67,1,0,8482,0.500000,63.099998,38,1739,23,0.693991,9369,0.694634,1025,0.611832,3161,0.643505,6174,0.609568,41573,1,71,0.789838,0.444840,0.020943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19999995,25,2760,18,1,2,143,4,0,3335,0.899902,31.100000,35,8579,17,0.645573,3366,0.809622,6319,0.728653,527,0.720728,1264,0.693247,15279,1,134,0.825527,0.357594,0.020232
19999996,33,517,9,1,5,31,1,0,532,0.399902,48.000000,32,54295,35,0.528440,545,0.337379,6509,0.522177,496,0.523364,535,0.574895,33511,0,62,0.080339,0.409130,0.020655
19999997,69,381,15,1,2,153,5,0,369,0.600098,88.599998,26,56638,19,0.744361,399,0.363448,19805,0.899083,109,0.858268,127,0.720505,29370,1,35,0.332007,0.559108,0.020900
19999998,649383,121,38,1,3,157,3,0,249,0.799805,578431.812500,35,172126,20,0.727778,360,0.906717,3677,0.583333,24,0.699454,183,0.687078,17085,0,102,0.977524,0.452832,0.021403


In [ ]:
vdf

,ts_delta,task_container_id,prior_question_elapsed_time,prior_question_had_explanation,part,t1,num_tag,u_attempt_c,total_explained,10_recent_correctness,10_recent_mean_gap,mean_elapsed,mean_gap,this_bundle_elapsed_time_mean,u_chance,u_attempts,c_chance,c_attempts,u_part_chance,u_part_attempts,u_skill_chance,u_skill_attempts,t_chance,t_attempts,prev_cor,prev_t1,trueskill_probability,mu,sigma
0,22,824,11,1,2,149,4,0,876,0.899902,20.799999,22,961,18,0.695426,962,0.860809,6157,0.838906,329,0.776139,746,0.731546,59637,1,142,0.887880,0.355988,0.020836
1,274,92,17,1,2,173,5,0,74,0.600098,149.000000,49,4683,16,0.596154,104,0.859472,41902,0.500000,8,0.475000,40,0.763955,30718,0,153,0.903571,0.376694,0.027779
2,55,1019,22,1,2,142,5,0,1897,0.700195,47.799999,50,29398,18,0.699064,1924,0.613618,35408,0.621951,82,0.690789,912,0.689115,3627,0,130,0.730812,0.498266,0.021121
3,87,190,26,1,5,54,1,0,265,0.700195,120.199997,58,137115,23,0.610714,280,0.838405,3286,0.460526,76,0.563380,142,0.628441,50921,1,57,0.935992,0.425220,0.021056
4,56,574,17,1,5,40,1,0,525,0.600098,4998.100098,19,11757,20,0.558659,537,0.347970,3325,0.492308,325,0.492308,325,0.579607,21485,0,81,0.109014,0.414726,0.019898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24271295,209,2301,83,1,2,146,4,1,2861,0.300049,285.200012,31,38652,18,0.610860,3094,0.468856,883,0.733906,466,0.693023,1290,0.709580,33242,1,53,0.302929,0.438708,0.020118
24271296,138,2302,22,1,5,94,1,1,2862,0.199951,267.600006,31,38635,27,0.610662,3095,0.755596,6567,0.520559,1289,0.552106,1804,0.707876,11405,0,146,0.858226,0.437576,0.020121
24271297,48084,2303,43,1,5,87,1,1,2863,0.300049,5044.600098,31,38639,22,0.610788,3096,0.747890,10309,0.520930,1290,0.552355,1805,0.708985,25742,1,94,0.849028,0.438167,0.020143
24271298,48198,2304,41,1,5,52,1,2,2864,0.300049,9833.000000,31,38643,32,0.610591,3097,0.337871,17116,0.520527,1291,0.552049,1806,0.612057,27942,0,87,0.103348,0.434652,0.020107


In [ ]:
tdf = tdf.to_numpy()
vdf = vdf.to_numpy()

In [ ]:
lgb_train = lgb.Dataset(tdf, y_train)
lgb_valid = lgb.Dataset(vdf, y_val)
del tdf, y_train
del vdf, y_val
gc.collect()

170

In [ ]:
params = {
    'objective': 'binary',
    'seed': 1103,
    'metric': ['auc'],
    'max_bin': 255,
    'learning_rate': 0.01,
    'num_leaves': 900,
    'min_data_in_leaf': 255,
#     'min_child_weight': 45.98,
#     'min_split_gain': 0.00001,
#     'lambda_l1':3.791,
#     'lambda_l2': 1.288,
#     'first_metric_only': True,
    'bagging_fraction': 0.7,
    'feature_fraction': 0.7,
    'device_type': 'gpu',
    'gpu_use_dp': True,
    'gpu_platform_id': 0,
    'gpu_device_id': 0,
}

model = lgb.train(params, 
                  lgb_train,
                  valid_sets=[lgb_valid],
                  verbose_eval=10,
                  num_boost_round=10000,
                  early_stopping_rounds=20,
                  feature_name = feat_cols,
                  categorical_feature = cat_cols,
                  init_model = '/content/drive/MyDrive/data/lgbm_model_new_mu_sigma_hung_trueskill_mean.txt'
                 )

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1555: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['num_tag', 'part', 'prev_cor', 'prev_t1', 'prior_question_had_explanation', 't1']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Info] Number of positive: 11422446, number of negative: 5577554
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5417
[LightGBM] [Info] Number of data points in the train set: 17000000, number of used features: 29
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 16
[LightGBM] [Info] 27 dense feature groups (453.95 MB) transferred to GPU in 0.373785 secs. 1 sparse feature groups


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 20 rounds
[4340]	valid_0's auc: 0.787189
[4350]	valid_0's auc: 0.787231
[4360]	valid_0's auc: 0.787271
[4370]	valid_0's auc: 0.787301
[4380]	valid_0's auc: 0.787334
[4390]	valid_0's auc: 0.787362
[4400]	valid_0's auc: 0.787389
[4410]	valid_0's auc: 0.787413
[4420]	valid_0's auc: 0.787442
[4430]	valid_0's auc: 0.787459
[4440]	valid_0's auc: 0.787479
[4450]	valid_0's auc: 0.787496
[4460]	valid_0's auc: 0.787516
[4470]	valid_0's auc: 0.787534
[4480]	valid_0's auc: 0.787556
[4490]	valid_0's auc: 0.787573
[4500]	valid_0's auc: 0.787589
[4510]	valid_0's auc: 0.787606
[4520]	valid_0's auc: 0.787617
[4530]	valid_0's auc: 0.787634
[4540]	valid_0's auc: 0.787647
[4550]	valid_0's auc: 0.787662
[4560]	valid_0's auc: 0.787679
[4570]	valid_0's auc: 0.787689
[4580]	valid_0's auc: 0.787704
[4590]	valid_0's auc: 0.787715
[4600]	valid_0's auc: 0.787724
[4610]	valid_0's auc: 0.787735
[4620]	valid_0's auc: 0.787743
[4630]	valid_0's auc: 0.787752
[4640]	va

In [ ]:
print('auc:', model.best_score['valid_0']['auc'])

auc: 0.7881925918642954


In [ ]:
model.save_model("/content/drive/MyDrive/data/lgbm_model_3rd_train_new_mu_sigma_hung_trueskill_mean.txt")